<a href="https://colab.research.google.com/github/Remi-Branco/Public/blob/master/Multithreading_decorator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##The multithreading decorator

In [ ]:
from functools import wraps
import os

def asynchronise(*args,**kwargs):
    """
    This decorator lets you multithread any function by passing it the list you wish to MT
    """
    
    verbose = False
    n_workers = None
    variable_to_asynchronise = None 

    for k,v in enumerate(kwargs):
        if v == "n_workers":
            n_workers = kwargs[v]
        elif v == "verbose":
            verbose = kwargs[v]
        else :
            variable_to_asynchronise = kwargs[v]

    if variable_to_asynchronise == None: #in case the variable was passed as an arg
        variable_to_asynchronise =  args[0]

    if n_workers == None:
        n_workers = len(variable_to_asynchronise)

    if verbose :
        print(f" Using {n_workers} workers")
        print(kwargs)
        print(variable_to_asynchronise)
    
    def decorator(function_to_decorate):

        @wraps(function_to_decorate)
        def wrapper(*args,**kwargs):
            result = []
            with ThreadPoolExecutor(max_workers=n_workers) as e:
                futures = {e.submit(function_to_decorate,var,*args,**kwargs): var for var in variable_to_asynchronise}
                for future in concurrent.futures.as_completed(futures):
                    result.append(future.result())
            return result    
        return wrapper
    return decorator


In [ ]:
%%time

#let's decorate a function
to_async = [1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2]


@asynchronise(to_async)
def waiting_function(t,call): #**kwargs was added to allow for any number of parameters to be passed
    """
    function that sleeps for t seconds, call is here to illustrate how to send arguments to the function
    """ 
    ##do stuff here
    time.sleep(t)
    ##do more stuff
    return t

results = waiting_function(call="Test" )
print(results)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]
CPU times: user 16.5 ms, sys: 1.31 ms, total: 17.8 ms
Wall time: 2.01 s


more parameters can be passed:
*  verbose
* n_workers



In [ ]:
%%time
@asynchronise(to_async, n_workers=4)
def waiting_function(t,call): #**kwargs was added to allow for any number of parameters to be passed
    """
    function that sleeps for t seconds, call is here to illustrate how to send arguments to the function
    """ 
    ##do stuff here
    time.sleep(t)
    ##do more stuff
    return t

results = waiting_function(call="Test" )
print(results)

[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 6.01 s
